# Initialize

In [ ]:
from pynq import Overlay
from pynq.drivers.video import HDMI
from pynq.board import Button

# Download bitstream
Overlay("base.bit").download()

# Initialize HDMI
hdmi_in = HDMI('in')
hdmi_out = HDMI('out')

# Initialize Button
button = Button(0)

# Start Stream from HDMI IN to OUT

In [ ]:
hdmi_in = HDMI('in', init_timeout = 10, frame_list=hdmi_out.frame_list)
hdmi_out.mode(4)
hdmi_in.start()
hdmi_out.start()

# Put HDMI IN in Another Buffer

In [ ]:
hdmi_in.frame_index_next()

# Video Processing & Output to HDMI OUT

In [ ]:
from IPython.display import Image
from PIL import Image as PIL_Image
import numpy as np

orig_img_path = '/home/xilinx/jupyter_notebooks/Examples/data/pokeball.png'
pokeball = PIL_Image.open(orig_img_path)
s_img = np.array(pokeball).reshape(100,100,4)

In [ ]:
import cv2
import numpy as np
import math

while(hdmi_in.state() == 1):
    frame = hdmi_in.frame_raw()
    img = (np.frombuffer(frame, dtype=np.uint8)).reshape(1080,1920,3)
    
    cv2.rectangle(img,(700,700),(400,400),(0,255,0),0)
    crop_img = img[400:700, 400:700]
    
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    (version, _, _) = cv2.__version__.split('.')

    if version is '3':
        image, contours, hierarchy = cv2.findContours(thresh1.copy(), \
               cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    elif version is '2':
        contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
               cv2.CHAIN_APPROX_NONE)

    cnt = max(contours, key = lambda x: cv2.contourArea(x))
    
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
    hull = cv2.convexHull(cnt)
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
    cv2.drawContours(drawing,[hull],0,(0,0,255),0)
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)

    
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img,far,1,[0,0,255],-1)
        cv2.line(crop_img,start,end,[0,255,0],2)

        
    if count_defects == 1:
        cv2.putText(img,"Please give us a good grade", (350,350), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 2:
        str = "This is a program that recognize hand gesture and put a pokeball in middle of your hand"
        cv2.putText(img, str, (350,350), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    elif count_defects == 3:
        cv2.putText(img,"Hope you enjoy", (350,350), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 5:
        x_offset= x + 450
        y_offset= y + 450
        for c in range(0,3):
            img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], c] = s_img[:,:,c] * (s_img[:,:,3]/255.0) +  img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], c] * (1.0 - s_img[:,:,3]/255.0)
    else:
        cv2.putText(img,"Please put your hand into the frame", (350,350),\
                    cv2.FONT_HERSHEY_SIMPLEX, 2, 2)

    hdmi_out.frame_raw(bytearray(img.tobytes()))
    
    if button.read():
        break

# Check State

In [ ]:
stateout = hdmi_out.state()
statein = hdmi_in.state()
width = hdmi_in.frame_width()
height = hdmi_in.frame_height()

print('Input resolution: ')
print('{}x{}'\
      .format(width,height))
print('Input state: ')
print(statein)
print('')
print('Outout resolution: ')
print(hdmi_out.mode())
print('Output state: ')
print(stateout)

# Stop Everything

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_out
del hdmi_in